In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from biolearn.data_library import DataLibrary

In [199]:
man=pd.read_table('../Blood-DNA-Methylation-Atlas/ref/HM450.hg38.manifest.tsv',index_col='Probe_ID')[['CpG_chrm','CpG_beg']]
man=man.dropna().copy()
man['CpG_beg']=man['CpG_beg'].astype(int)
man.columns=[0,1]
man['cg']=man.index

In [3]:
%%time 
wb_='data/cell/GSE41169_series_matrix.txt'
age=pd.read_table(wb_,skiprows=32+14,nrows=0,index_col=0).columns.str.strip('age: ').astype(float).tolist()
wb=pd.read_table(wb_,skiprows=32+14+26,index_col=0).drop('!series_matrix_table_end').round(3)

CPU times: user 7.6 s, sys: 936 ms, total: 8.53 s
Wall time: 8.61 s


In [4]:
%%time 
wb_t=wb.T
wb_t['age']=age
wb_cor=wb_t.corrwith(wb_t['age']).drop('age').sort_values()

CPU times: user 2min 2s, sys: 147 ms, total: 2min 2s
Wall time: 2min 2s


In [200]:
man=man.merge(wb_cor.rename('cor'),left_index=True,right_index=True)
man=man[man[0].isin('chr'+str(c) for c in range(1,23))].copy()
man['pos']=man[0].str[3:].astype(int)*10**9+man[1]
man=man.sort_values('pos')

In [ ]:
hmr=pd.read_table('/labs/mpsnyder/moqri/epi_deconv/data/arc/hmr/neo',header=None,index_col=3,usecols=[0,1,2,3])
hmr['b']=hmr[0].str[3:].astype(int)*10**9+hmr[1]
hmr['e']=hmr[0].str[3:].astype(int)*10**9+hmr[2]
hmr=hmr.sort_values('b')

In [201]:
%%time
l=[]
cur=0
for i in range(len(man)):
    if cur>=len(hmr):
        l.append(-1)
    else: 
        pos=man['pos'].iloc[i]
        cg=man['cg'].iloc[i]
        b=hmr['b'].iloc[cur]
        e=hmr['e'].iloc[cur]        
        if pos<b:
            l.append(-1)
        if pos>=b and pos<=e:
            l.append(cur)
        if pos>e:
            l.append(-1)
            cur+=1

CPU times: user 22.4 s, sys: 19.7 ms, total: 22.5 s
Wall time: 22.5 s


In [202]:
man['l']=l

In [203]:
mg=pd.DataFrame(man.groupby('l')[0].count())
mg=mg[mg[0]>=5].copy()
print(len(mg))
mg['cor']=man.groupby('l')['cor'].mean()

14855


In [204]:
hmr.iloc[mg.sort_values('cor').tail(10).index][[0,1,2]]

,0,1,2
3,,,
HYPO18969,chr17,28985870,28986481
HYPO43087,chr8,56447897,56448292
HYPO23856,chr19,41996936,41999519
HYPO24496,chr19,51887336,51888227
HYPO24741,chr19,57708673,57709644
HYPO36296,chr5,93620363,93621878
HYPO23522,chr19,35755461,35756232
HYPO1989,chr1,52842501,52844181
HYPO2189,chr1,68046821,68047168


In [206]:
man=man[man['l']>0].copy()

In [207]:
man['bin']=man['cor']>0

In [208]:
%%time 
cont=[]
c=0
last=False
last_l=-1
for i,b in man[['l','bin']].iterrows():
    if (b['bin']):
        if (b['l']>last_l or last==False):
            c+=1
            last_l=b['l']
        cont.append(c)
        last=True  
    else:
        cont.append(-1)
        last=False

CPU times: user 11.7 s, sys: 18.8 ms, total: 11.7 s
Wall time: 11.7 s


In [209]:
man['cont']=cont

In [210]:
mgc=pd.DataFrame(man.groupby('cont')[0].count())
mgc['chr']=man.groupby('cont')[0].first()
mgc['b']=man.groupby('cont')[1].first()
mgc['e']=man.groupby('cont')[1].last()
mgc['cor']=man.groupby('cont')['cor'].mean()

In [211]:
topl=mgc[mgc[0]>=5].sort_values('cor')

In [212]:
topl.tail(10)

,0,chr,b,e,cor
cont,,,,,
41084,7,chr19,35755493,35756004,0.481751
30316,7,chr13,30905990,30906525,0.490382
42916,5,chr20,21391487,21392168,0.509066
16728,6,chr6,117481424,117481998,0.509642
34070,6,chr16,693924,694938,0.532847
1994,6,chr1,52842924,52843588,0.533822
7722,6,chr2,218872669,218874009,0.552954
2214,5,chr1,68046855,68047161,0.581576
16807,5,chr6,127514604,127515013,0.624143


,0,chr,b,e,cor
cont,,,,,
102354,11,chr19,57708711,57709468,0.475737
23123,7,chr3,158094228,158095627,0.476681
5060,10,chr1,68046855,68049525,0.478658
74531,7,chr13,30905990,30906525,0.490382
17184,7,chr2,218872669,218874841,0.495426
103079,5,chr20,21391487,21392168,0.509066
40364,6,chr6,117481424,117481998,0.509642
83880,6,chr16,693924,694938,0.532847
40519,5,chr6,127514604,127515013,0.624143


In [ ]:
%%time
mono_='data/cell/GSE56046_methylome_normalized.txt'
mono=pd.read_table(mono_,index_col=0,usecols=[0]+list(range(1,201,2)))
p2=pow(2,mono).round(3)
mono=p2/(p2+1)
age=pd.read_table('https://ftp.ncbi.nlm.nih.gov/geo/series/GSE56nnn/GSE56046/matrix/GSE56046_series_matrix.txt.gz',skiprows=46,
             index_col=0).iloc[10].str.strip('age (yrs): ').astype(int)
mono_t=mono.T
mono_t['age']=age.head(100).values
mono_cor=mono_t.corrwith(mono_t['age']).drop('age').sort_values()

In [ ]:
%%time
t_='data/cell/GSE56581_methylome_normalized.txt'
t=pd.read_table(t_,index_col=0,usecols=[0]+list(range(1,201,2)))
p2=pow(2,t).round(3)
t=p2/(p2+1)
age=pd.read_table('https://ftp.ncbi.nlm.nih.gov/geo/series/GSE56nnn/GSE56581/matrix/GSE56581_series_matrix.txt.gz',skiprows=40,
             index_col=0).iloc[10].str.strip('age (yrs): ').astype(int)
t_t=t.T
t_t['age']=age.head(100).values
t_cor=t_t.corrwith(t_t['age']).drop('age').sort_values()

In [ ]:
%%time
liver_='data/cell/GSE48325_series_matrix.txt'
liver=pd.read_table(liver_,skiprows=27+33,index_col=0).round(3)
age=pd.read_table(liver_,skiprows=27,nrows=50,index_col=0).iloc[8].str.strip('age: ').astype(int)
liver_t=liver.T
liver_t['age']=age.values
liver_cor=liver_t.corrwith(liver_t['age']).drop('age').sort_values()

In [ ]:
dd=pd.concat([wb_cor,t_cor,mono_cor,liver_cor],axis=1).drop('!series_matrix_table_end')
dd.corr()

In [ ]:
t_hm=pd.read_table('../Blood-DNA-Methylation-Atlas/data/merge/hm450/cd4.meth',header=None,skiprows=860,index_col=3)[[0,1,4]]
m_hm=pd.read_table('../Blood-DNA-Methylation-Atlas/data/merge/hm450/mono.meth',header=None,skiprows=860,index_col=3)[[0,1,4]]